The Plotly pane displays [Plotly plots](https://plotly.com/python/) within a Panel application. It enhances the speed of plot updates by using binary serialization for array data contained in the Plotly object. 

Please remember that to use the Plotly pane in a Jupyter notebook, you must activate the Panel extension and include `"plotly"` as an argument. This step ensures that plotly.js is properly set up.

In [ ]:
import panel as pn

pn.extension("plotly")

## Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

### Core

* **``object``** (object): The Plotly `Figure` or dictionary object being displayed.
* **``config``** (dict): Additional configuration of the plot. See [Plotly configuration options](https://plotly.com/javascript/configuration-options/).

### Callbacks

* **``click_data``** (dict): Click callback data
* **``clickannotation_data``** (dict): Clickannotation callback data
* **``hover_data``** (dict): Hover callback data
* **``link_figure``** (bool): Attach callbacks to the Plotly figure to update output when it is modified in place.
* **``relayout_data``** (dict): Relayout callback data
* **``restyle_data``** (dict): Restyle callback data
* **``selected_data``** (dict): Selected callback data
* **``viewport``** (dict): Current viewport state, i.e. the x- and y-axis limits of the displayed plot.
* **``viewport_update_policy``** (str, default = 'mouseup'): Policy by which the viewport parameter is updated during user interactions
    * ``mouseup``: updates are synchronized when mouse button is released after panning
    * ``continuous``: updates are synchronized continually while panning
    * ``throttle``: updates are synchronized while panning, at intervals determined by the viewport_update_throttle parameter
* **``viewport_update_throttle``** (int, default = 200, bounds = (0, None)): Time interval in milliseconds at which viewport updates are synchronized when viewport_update_policy is "throttle".

___

As with most other types ``Panel`` will automatically convert a Plotly `Figure` to a `Plotly` pane if it is passed to the `pn.panel` function or a Panel layout, but a `Plotly` pane can also be constructed directly using the `pn.pane.Plotly` constructor:

In [ ]:
import numpy as np
import plotly.graph_objs as go

import panel as pn

pn.extension("plotly")

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))

surface=go.Surface(z=z)
fig = go.Figure(data=[surface])

fig.update_layout(
    title="Plotly 3D Plot",
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50),
)

plotly_pane = pn.pane.Plotly(fig)
plotly_pane

Once created `Plotly` pane can be updated by assigning a new figure object

In [ ]:
new_fig = go.Figure(data=[go.Surface(z=np.sin(z+1))])
new_fig.update_layout(
    title="Plotly 3D Plot",
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50),
)

plotly_pane.object = new_fig

Lets reset the Plotly pane

In [ ]:
plotly_pane.object = fig

## Efficient Updates with Dictionaries

Instead of updating the entire Figure you can efficiently update specific traces or the layout if you use a dictionary instead of a Plotly Figure.

Note updates will only be fast if the ``Figure`` is defined as a dictionary, since Plotly will make copies of the traces, which means that modifying them in place has no effect. Modifying an array will send just the array using a binary protocol, leading to fast and efficient updates.

In [ ]:
import numpy as np
import plotly.graph_objs as go

import panel as pn

pn.extension("plotly")

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))

surface = go.Surface(z=z)
layout = go.Layout(
    title='Plotly 3D Plot',
    autosize=False,
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50)
)

fig = dict(data=[surface], layout=layout)

plotly_pane = pn.pane.Plotly(fig)
plotly_pane

In [ ]:
surface.z = np.sin(z+1)
plotly_pane.object = fig

Similarly, modifying the plot ``layout`` will only modify the layout, leaving the traces unaffected.

In [ ]:
fig['layout']['width'] = 800

plotly_pane.object = fig

Lets reset the Plotly pane

In [ ]:
surface.z = z
fig['layout']['width'] = 500

plotly_pane.object = fig

## Plotly Layouts

The `Plotly` pane supports layouts and subplots of arbitrary complexity, allowing even deeply nested Plotly figures to be displayed:

In [ ]:
import panel as pn
from plotly import subplots

pn.extension("plotly")

heatmap = go.Heatmap(
    z=[[1, 20, 30],
       [20, 1, 60],
       [30, 60, 1]],
    showscale=False)

y0 = np.random.randn(50)
y1 = np.random.randn(50)+1

box_1 = go.Box(y=y0)
box_2 = go.Box(y=y1)
data = [heatmap, box_1, box_2]

fig = subplots.make_subplots(
    rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
    subplot_titles=('First Subplot','Second Subplot', 'Third Subplot')
)

fig.append_trace(box_1, 1, 1)
fig.append_trace(box_2, 1, 2)
fig.append_trace(heatmap, 2, 1)

fig['layout'].update(height=600, width=600, title='i <3 subplots')

# Conversion to dictionary only necessary if you want efficient updates
fig = fig.to_dict()

subplot_panel = pn.pane.Plotly(fig)
subplot_panel

Just like in the single-subplot case we can modify just certain aspects of a plot and then trigger an update. E.g. here we replace the overall title text:

In [ ]:
fig['layout']['title']['text'] = 'i <3 updating subplots'
subplot_panel.object = fig

Lets reset the plot

In [ ]:
fig['layout']['title']['text'] = 'i <3 subplots'
subplot_panel.object = fig

## Responsive Plots

Plotly plots can be made responsive using the `autosize` option on a Plotly layout and a responsive `sizing_mode` argument to the `Plotly` pane:

In [ ]:
import pandas as pd
import panel as pn
import plotly.express as px

pn.extension("plotly")

data = pd.DataFrame([
    ('Monday', 7), ('Tuesday', 4), ('Wednesday', 9), ('Thursday', 4),
    ('Friday', 4), ('Saturday', 4), ('Sunday', 4)], columns=['Day', 'Orders']
)

fig = px.line(data, x="Day", y="Orders")
fig.update_traces(mode="lines+markers", marker=dict(size=10), line=dict(width=4))
fig.layout.autosize = True

responsive = pn.pane.Plotly(fig, height=300)

pn.Column('## A responsive plot', responsive, sizing_mode='stretch_width')

## Plot Configuration

You can set the [Plotly configuration options](https://plotly.com/javascript/configuration-options/) via the `config` parameter. Lets try to configure `scrollZoom`:

In [ ]:
responsive = pn.pane.Plotly(fig, config={"scrollZoom": True}, height=300)

pn.Column('## A responsive and zoomable plot', responsive, sizing_mode='stretch_width')

## Controls

The `Plotly` pane exposes a number of options which can be changed from both Python and Javascript try out the effect of these parameters interactively:

In [ ]:
pn.Row(responsive.controls(jslink=True), responsive)